# Connect to Milvus

In [2]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema
)

## Techzone's Standalone Milvus instance
# host = '161.156.196.183'
# port = '8080'
# password = '4XYg2XK6sMU4UuBEjHq4EhYE8mSFO3Qq'
# user = 'root'
# server_pem_path =  './data/cert.pem'
# server_name = 'localhost'
## Techzone's watsonx.data Milvus service
host = 'useast.services.cloud.techzone.ibm.com'
port = '28048'
password = 'password'
user = 'ibmlhadmin'
server_pem_path = './data/milvus_cert.pem'
server_name = 'watsonxdata'

connections.connect(alias = 'default',
                host = host,
                port = port,
                user = user,
                password = password,
                server_pem_path=server_pem_path,
                server_name = server_name,
                secure = True)

In [3]:
connections.list_connections()

[('default', <pymilvus.client.grpc_handler.GrpcHandler at 0x22652979ee0>)]

# Creating Milvus schema definition

## News collections

In [6]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=2500,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name="school_year", dtype=DataType.INT32),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="file_links", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
]

schema = CollectionSchema(fields, "news schema")

In [7]:
collection_names = ['recruitment', 'timetable', 'scholarship', 'academic_affairs', 'events']

Add search index

In [8]:
for col in collection_names:
    collection = Collection(col, schema)
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name='embedding', index_params=index_params)

## For student handbook collections (default collections)

In [9]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300,),
    FieldSchema(name="article", dtype=DataType.VARCHAR, max_length=5000,),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024,),
    FieldSchema(name="page_number", dtype=DataType.INT32),
    FieldSchema(name="school_year", dtype=DataType.INT32,),
    FieldSchema(name="in_effect", dtype=DataType.VARCHAR, max_length=100,),
    FieldSchema(name="created_at", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="updated_at", dtype=DataType.VARCHAR, max_length=200,),
] #Missing page number

schema = CollectionSchema(fields, "news schema")

collection = Collection('student_handbook', schema)

index_params = {
    'metric_type':'L2',
    'index_type':"IVF_FLAT",
    'params':{"nlist":2048}
}
collection.create_index(field_name='embedding', index_params=index_params)

Status(code=0, message=)

## Check collection

In [11]:
connections._fetch_handler('default').list_collections()

['timetable',
 'events',
 'student_handbook',
 'recruitment',
 'scholarship',
 'academic_affairs']

UTILS: drop all collections

In [5]:
for col in connections._fetch_handler('default').list_collections():
    connections._fetch_handler('default').drop_collection(col)

# Process data
- Embedding content
- Preprocess headings
- Add columns
- Remove nulls
- Reformat

## Initialize the embedding model

Import

In [95]:
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.foundation_models import Embeddings
from dotenv import load_dotenv

Credentials & initialize

In [96]:
load_dotenv('./data/.env')
my_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.environ['WATSONX_APIKEY'],
}

# model_id = 'sentence-transformers/all-minilm-l12-v2'
model_id = 'intfloat/multilingual-e5-large'
gen_parms = None
project_id = os.environ['WATSONX_PROJECT_ID']
space_id = None
verify = False

# Set the truncate_input_tokens to a value that is equal to or 
# less than the maximum allowed tokens for the embedding model that you are using. 
# If you don't specify this value and the input has more tokens than the model can process, 
# an error is generated.

embed_params = {
    EmbedParams.TRUNCATE_INPUT_TOKENS: 512,
}

model = Embeddings(
    model_id=model_id,
    credentials=my_credentials,
    params=embed_params,
    project_id=project_id,
    verify=verify
)

Preprocess function
- As mentioned from [the model card](https://huggingface.co/intfloat/multilingual-e5-large), it is advised to format the *to-be-embed* content as following `passage:...` and/or `query:...`

In [97]:
def preprocess_embed(text, isQuery=False):
    if isQuery:
        return "query: " + text
    else: #embed content
        return "passage: " + text

## DATA: Student handbook

In [98]:
import pandas as pd
from glob import glob

paths = glob('./data/student_handbook/*')

Many unicode characters when extracted from the student handbook are incorrectly formatted *(probably on purpose)*

So we have a function to reformat these unicode characters.

In [99]:
# Function to replace /uniXXXX with the corresponding Unicode character
import re

def replace_unicode_references(text):
    # Use regex to match the '/uniXXXX' sequences
    result = re.sub(r'/uni([0-9A-Fa-f]{4})', lambda m: chr(int(m.group(1), 16)), text)
    result = result.replace('f_', 'f')
    return result

In [100]:
from pypdf import PdfReader
from tqdm.notebook import tqdm

data = []
for path in tqdm(paths, desc="Documents", position=0):
    reader = PdfReader(path)
    n_pages = len(reader.pages)

    title = path.split('\\')[-1].split('.')[0]
    for i, page in tqdm(enumerate(reader.pages), desc="Pages", position=1, leave=False):
        text = page.extract_text() #Article
        text = replace_unicode_references(text)
        embed_text = preprocess_embed(title + '\n' + text) 
        embedding = model.embed_query(embed_text) #Embedding
        
        page_number = i + 1 #Page number
        school_year = 2024 #School year
        in_effect = 2024 #In effect till

        record = {
            'title': title,
            'article': text,
            'embedding': embedding,
            'page_number': page_number,
            'school_year': school_year,
            'in_effect': in_effect,
            'created_at': '',
            'updated_at': ''
        }
        data.append(record)


Documents:   0%|          | 0/5 [00:00<?, ?it/s]

Pages: 0it [00:00, ?it/s]

Pages: 0it [00:00, ?it/s]

Pages: 0it [00:00, ?it/s]

Pages: 0it [00:00, ?it/s]

Pages: 0it [00:00, ?it/s]

In [101]:
len(data)

129

In [102]:
print(data[0])

{'title': 'Khoa công nghệ thông tin', 'article': 'KHOA \nCÔNG NGHỆ THÔNG TIN\nThông tin chung\ninfo@/fit.hcmus.edu.vn\nHỗ trợ sinh viên: tlsv@/fit.hcmus.edu.vn Giáo vụ: o Chương trình Chuẩn: giaovu@/fit.hcmus.edu.vn o Chương trình Chất lượng cao: giaovu.clc@/fit.hcmus.edu.vno Chương trình Tiên tiến: giaovu@apcs./fitus.edu.vnCố vấn học tập: cvht@/fit.hcmus.edu.vn Chương trình đề án: ctdb@hcmus.edu.vn(028) 38 354 266 (Ext: 500)(028) 62 884 499 (Ext: 4000)Văn phòng khoa: phòng I.53, 227 Nguyễn Văn Cừ, Q.5, TP .HCM\nwww./fit.hcmus.edu.vnBan Chủ nhiệm:Trưởng Khoa: TS. Đinh Bá TiếnPhó Trưởng Khoa: TS. Lâm Quang Vũ, PGS.TS. Nguyễn Văn Vũ, ThS. Văn Chí Nam \nMỤC TIÊU ĐÀO TẠO\nSỨ MỆNH\n- Cung cấp các trải nghiệm giảng dạy và học tập hàng đầu cho các chương trình đào tạo bậc đại học và sau đại học trong lĩnh vực máy tính và công nghệ thông tin.- Đào tạo sinh viên, học viên trở thành những nhà phát triển giải pháp công nghệ thông tin hoặc trở thành lãnh đạo chuyên nghiệp, thành công, tự quyết và 

In [107]:
import json

with open('student_handbook_embedded.json', 'w+') as wstream:
    json.dump(data, wstream, indent=1)

In [103]:
df = pd.DataFrame(data)

In [105]:
df.to_csv('student_handbook_embedded.csv', index=False)

In [90]:
from pypdf import PdfReader

path = r".\data\student_handbook\Khoa công nghệ thông tin.pdf"
reader = PdfReader(path)

text = reader.pages[0].extract_text()
text

'KHOA \nCÔNG NGH/uni1EC6 THÔNG TIN\nThông tin chung\ninfo@/f_it.hcmus.edu.vn\nH/uni1ED7 tr/uni1EE3 sinh viên: tlsv@/f_it.hcmus.edu.vn Giáo v/uni1EE5: o Chương trình Chu/uni1EA9n: giaovu@/f_it.hcmus.edu.vn o Chương trình Ch/uni1EA5t lư/uni1EE3ng cao: giaovu.clc@/f_it.hcmus.edu.vno Chương trình Tiên ti/uni1EBFn: giaovu@apcs./f_itus.edu.vnC/uni1ED1 v/uni1EA5n h/uni1ECDc t/uni1EADp: cvht@/f_it.hcmus.edu.vn Chương trình đ/uni1EC1 án: ctdb@hcmus.edu.vn(028) 38 354 266 (Ext: 500)(028) 62 884 499 (Ext: 4000)Văn phòng khoa: phòng I.53, 227 Nguy/uni1EC5n Văn C/uni1EEB, Q.5, TP .HCM\nwww./f_it.hcmus.edu.vnBan Ch/uni1EE7 nhi/uni1EC7m:Trư/uni1EDFng Khoa: TS. Đinh Bá Ti/uni1EBFnPhó Trư/uni1EDFng Khoa: TS. Lâm Quang Vũ, PGS.TS. Nguy/uni1EC5n Văn Vũ, ThS. Văn Chí Nam \nM/uni1EE4C TIÊU ĐÀO T/uni1EA0O\nS/uni1EE8 M/uni1EC6NH\n- Cung c/uni1EA5p các tr/uni1EA3i nghi/uni1EC7m gi/uni1EA3ng d/uni1EA1y và h/uni1ECDc t/uni1EADp hàng đ/uni1EA7u cho các chương trình đào t/uni1EA1o b/uni1EADc đ/uni1EA1i h/uni1ECDc

In [77]:
text = text.replace('/uni', r'\u')

In [91]:
import re

# Original string with broken Unicode references
broken_string = """- Ki/uni1EBFn th/uni1EE9c n/uni1EC1n t/uni1EA3ng v/uni1EC1 khoa h/uni1ECDc: Kh/uni1ED1i \nki/uni1EBFn th/uni1EE9c v/uni1EC1 Toán, V/uni1EADt lý, Đi/uni1EC7n - Đi/uni1EC7n t/uni1EED.- Ki/uni1EBFn th/uni1EE9c n/uni1EC1n t/uni1EA3ng lĩnh v/uni1EF1c CNTT: Kh/uni1ED1i ki/uni1EBFn th/uni1EE9c v/uni1EC1 l/uni1EADp trình, ki/uni1EBFn th/uni1EE9c t/uni1ED5ng quát v/uni1EC1 lĩnh v/uni1EF1c CNTT.- Ki/uni1EBFn th/uni1EE9c k/uni1EF9 thu/uni1EADt nâng cao, các công c/uni1EE5 và phương pháp trong ngành CNTT: Kh/uni1ED1i ki/uni1EBFn th/uni1EE9c v/uni1EC1 c/uni1EA5u trúc d/uni1EEF li/uni1EC7u và gi/uni1EA3i thu/uni1EADt, kh/uni1ED1i ki/uni1EBFn th/uni1EE9c v/uni1EC1 đi/uni1EC1u hành máy tính, kh/uni1ED1i ki/uni1EBFn th/uni1EE9c v/uni1EC1 ki/uni1EBFn trúc máy tính, kh/uni1ED1i ki/uni1EBFn th/uni1EE9c v/uni1EC1 m/uni1EA1ng máy tính, cơ s/uni1EDF d/uni1EEF li/uni1EC7u, an ninh b/uni1EA3o m/uni1EADt tính riêng tư và các công c/uni1EE5, phương pháp và công ngh/uni1EC7 h/uni1ED7 tr/uni1EE3 trong ngành CNTT.- Các ki/uni1EBFn th/uni1EE9c nâng cao c/uni1EE7a t/uni1EEBng chuyên ngành.- K/uni1EF9 năng và tính cách cá nhân (đ/uni1ED9c l/uni1EADp, t/uni1EF1 tin, s/uni1EB5n sàng ra quy/uni1EBFt đ/uni1ECBnh, sáng t/uni1EA1o, cách nghĩ mang tính ph/uni1EA3n bi/uni1EC7n, thích nghi môi trư/uni1EDDng m/uni1EDBi, qu/uni1EA3n lý tài nguyên cá nhân, h/uni1ECDc và t/uni1EF1 h/uni1ECDc, qu/uni1EA3n tr/uni1ECB d/uni1EF1 án…); - K/uni1EF9 năng nhóm (thành l/uni1EADp nhóm, ho/uni1EA1t đ/uni1ED9ng trong nhóm, lãnh đ/uni1EA1o nhóm, phát tri/uni1EC3n nhóm…);- K/uni1EF9 năng giao ti/uni1EBFp (trình bày, đàm phán, phát tri/uni1EC3n các m/uni1ED1i quan h/uni1EC7 xã h/uni1ED9i,...); - K/uni1EF9 năng ngo/uni1EA1i ng/uni1EEF (phát tri/uni1EC3n toàn di/uni1EC7n b/uni1ED1n k/uni1EF9 năng Nghe - Nói - Đ/uni1ECDc - Vi/uni1EBFt và kh/uni1EA3 năng s/uni1EED d/uni1EE5ng t/uni1ED1t các thu/uni1EADt ng/uni1EEF chuyên ngành); - K/uni1EF9 năng lãnh đ/uni1EA1o (thái đ/uni1ED9 lãnh đ/uni1EA1o, nh/uni1EADn bi/uni1EBFt các v/uni1EA5n đ/uni1EC1, s/uni1EF1 c/uni1ED1 và ngh/uni1ECBch lý, đ/uni1EC1 xu/uni1EA5t và sáng t/uni1EA1o trong vi/uni1EC7c gi/uni1EA3i quy/uni1EBFt các v/uni1EA5n đ/uni1EC1, s/uni1EF1 c/uni1ED1, xây d/uni1EF1ng và d/uni1EABn d/uni1EAFt m/uni1ED9t t/uni1ED5 ch/uni1EE9c, lên k/uni1EBF ho/uni1EA1ch và d/uni1EABn d/uni1EAFt d/uni1EF1 án đ/uni1EBFn thành công,...)- K/uni1EF9 năng kh/uni1EDFi nghi/uni1EC7p (thành l/uni1EADp công ty, t/uni1ED5 ch/uni1EE9c công ty và qu/uni1EA3n tr/uni1ECB, vi/uni1EBFt k/uni1EBF ho/uni1EA1ch kinh doanh, tài chính công ty, hình thành ý tư/uni1EDFng s/uni1EA3n ph/uni1EA9m, d/uni1ECBch v/uni1EE5 d/uni1EF1 trên công ngh/uni1EC7, sáng t/uni1EA1o trong s/uni1EA3n ph/uni1EA9m/ d/uni1ECBch v/uni1EE5,...)- Ng/uni1EEF c/uni1EA3nh bên ngoài, xã h/uni1ED9i, kinh t/uni1EBF và môi trư/uni1EDDng.- Ng/uni1EEF c/uni1EA3nh công ty và doanh nghi/uni1EC7p.- Đ/uni1EA1o đ/uni1EE9c, trách nhi/uni1EC7m và các giá tr/uni1ECB cá nhân c/uni1ED1t lõi.\n- Suy lu/uni1EADn có phân tích và gi/uni1EA3i quy/uni1EBFt v/uni1EA5n đ/uni1EC1.
- Th/uni1EF1c nghi/uni1EC7m, đi/uni1EC1u tra và khám phá tri th/uni1EE9c.- Suy nghĩ t/uni1EA7m m/uni1EE9c h/uni1EC7 th/uni1ED1ng.- Hình thành ý tư/uni1EDFng/ bài toán/ d/uni1EF1 án. - Thi/uni1EBFt k/uni1EBF h/uni1EC7 th/uni1ED1ng CNTT (gi/uni1EA3i pháp,s/uni1EA3n ph/uni1EA9m…). - Hi/uni1EC7n th/uni1EF1c hóa (implementation): các ti/uni1EBFn trình và phương pháp hi/uni1EC7n th/uni1EF1c hóa; hi/uni1EC7n th/uni1EF1c hóa h/uni1EC7 th/uni1ED1ng d/uni1EF1a trên thi/uni1EBFt k/uni1EBF… \nKI/uni1EBEN TH/uni1EE8C K/uni1EF8 NĂNG\nNG/uni1EEE C/uni1EA2NH, TRÁCH NHI/uni1EC6MVÀ Đ/uni1EA0O Đ/uni1EE8C\nPHƯƠNG PHÁP NGHIÊN C/uni1EE8U VÀ KHOA H/uni1ECCCHÌNH THÀNH Ý TƯ/uni1EDENG, THI/uni1EBET K/uni1EBEVÀ HI/uni1EC6N TH/uni1EF0C HÓA H/uni1EC6 TH/uni1ED0NG CNTT\nKI/uni1EC2M CH/uni1EE8NG, V/uni1EACN HÀNH, B/uni1EA2O TRÌVÀ PHÁT TRI/uni1EC2N H/uni1EC6 TH/uni1ED0NG CNTT\n19"""

# Function to replace /uniXXXX with the corresponding Unicode character
def replace_unicode_references(text):
    # Use regex to match the '/uniXXXX' sequences
    result = re.sub(r'/uni([0-9A-Fa-f]{4})', lambda m: chr(int(m.group(1), 16)), text)
    result = result.replace('f_', 'f')
    return result

# Replace the sequences in the broken string
fixed_string = replace_unicode_references(text)

fixed_string


'KHOA \nCÔNG NGHỆ THÔNG TIN\nThông tin chung\ninfo@/fit.hcmus.edu.vn\nHỗ trợ sinh viên: tlsv@/fit.hcmus.edu.vn Giáo vụ: o Chương trình Chuẩn: giaovu@/fit.hcmus.edu.vn o Chương trình Chất lượng cao: giaovu.clc@/fit.hcmus.edu.vno Chương trình Tiên tiến: giaovu@apcs./fitus.edu.vnCố vấn học tập: cvht@/fit.hcmus.edu.vn Chương trình đề án: ctdb@hcmus.edu.vn(028) 38 354 266 (Ext: 500)(028) 62 884 499 (Ext: 4000)Văn phòng khoa: phòng I.53, 227 Nguyễn Văn Cừ, Q.5, TP .HCM\nwww./fit.hcmus.edu.vnBan Chủ nhiệm:Trưởng Khoa: TS. Đinh Bá TiếnPhó Trưởng Khoa: TS. Lâm Quang Vũ, PGS.TS. Nguyễn Văn Vũ, ThS. Văn Chí Nam \nMỤC TIÊU ĐÀO TẠO\nSỨ MỆNH\n- Cung cấp các trải nghiệm giảng dạy và học tập hàng đầu cho các chương trình đào tạo bậc đại học và sau đại học trong lĩnh vực máy tính và công nghệ thông tin.- Đào tạo sinh viên, học viên trở thành những nhà phát triển giải pháp công nghệ thông tin hoặc trở thành lãnh đạo chuyên nghiệp, thành công, tự quyết và có đạo đức, đủ năng lực áp dụng kiến thức và kỹ n

## DATA: News data

In [1]:
import pandas as pd

df = pd.read_csv('./data/FIT_news.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        640 non-null    object
 1   article      640 non-null    object
 2   file_links   640 non-null    object
 3   url          640 non-null    object
 4   type         640 non-null    object
 5   created_at   640 non-null    object
 6   school_year  640 non-null    int64 
 7   in_effect    640 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 40.1+ KB


In [62]:
from glob import glob

paths = glob('./data/thesis_rag_data/*')

In [63]:
import json 
data = []
for path in paths:
    with open(path, 'r', encoding='utf-8') as rstream:
        data = json.load(rstream)

    for d in data:
        d.pop('_id', None)
        d['school_year'] = int(d['school_year'])
    collection = Collection(path.split('\\')[-1].split('.')[0])
    collection.insert(data)


In [ ]:
path = paths[4]
path.split('\\')[-1].split('.')[0]

In [7]:
import json
with open(path, 'r', encoding='utf-8') as rstream:
    data = json.load(rstream)

for d in data:
    d.pop('_id', None)
    d['school_year'] = int(d['school_year'])

In [ ]:
for col in collection_names:
    Collection(col).insert(data)

In [134]:
collection = Collection('timetable')

In [ ]:
collection.insert(data)

***

In [80]:
# for c in connections._fetch_handler('default').list_collections():
#     Collection(c).drop()

***

In [65]:
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.foundation_models import Embeddings
from dotenv import load_dotenv

In [66]:
load_dotenv('./data/.env')
my_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.environ['WATSONX_APIKEY'],
}

# model_id = 'sentence-transformers/all-minilm-l12-v2'
model_id = 'intfloat/multilingual-e5-large'
gen_parms = None
project_id = os.environ['WATSONX_PROJECT_ID']
space_id = None
verify = False

# Set the truncate_input_tokens to a value that is equal to or 
# less than the maximum allowed tokens for the embedding model that you are using. 
# If you don't specify this value and the input has more tokens than the model can process, 
# an error is generated.

embed_params = {
    EmbedParams.TRUNCATE_INPUT_TOKENS: 512,
}

model = Embeddings(
    model_id=model_id,
    credentials=my_credentials,
    params=embed_params,
    project_id=project_id,
    verify=verify
)

In [67]:
query="Khoa công nghệ thông tin là gì"

In [68]:
query_embeddings = model.embed_query(query)

col_name = "student_handbook"
collection = Collection(col_name)
collection.load()
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 5}
}
top_k = 3
output_fields = ['title', 'article'] #Section field to be added
results = collection.search(
    data=[query_embeddings],
    anns_field="embedding",
    param=search_params,
    limit=top_k,
    expr=None,
    output_fields=output_fields
)

In [ ]:
results[0][0].to_dict()